In [1]:
# Author: Meysam Aghighi
# Contact: meysam.aghighi@gmail.com/meysam.aghighi@ericsson.com
# Date: 2024-11-08
# Description: A script to automate desk booking in flowscape

import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime, timedelta

In [2]:
today = datetime.today()

building = "SEKI10 Kista"
floor = "Floor 8"
desk = "326A"
dates = [(today + timedelta(days=i)).day for i in range(16)]
# dates = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

# Load cookies from the saved JSON file
with open("flowscape_cookies.json", "r") as f:
    cookies = json.load(f)

# Open Chrome
chrome_options = Options() # this is important for cookies to work
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()

# Go to webpage
driver.get("https://ericsson.flowscape.se/webapp/")

# Add each cookie to the session
for cookie in cookies:    
#     cookie.pop('sameSite', None)
    driver.add_cookie(cookie)
    
driver.refresh()

# Select building
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f"//button[text()='{building}']"))
)
button.click()
# Select floor
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f"//button[text()='{floor}']"))
)
button.click()

print(f"searching the desk {desk}...")
# Search the desk
search_field = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Search']"))
)
search_field.send_keys(desk)
time.sleep(1)
print("click on the first search...")
# Open the first search (desk)
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'sc-qZtCU cJnHnZ')]"))
)
button.click()

div_element = WebDriverWait(driver, 10).until(
EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-pBxWu.gglDmq"))
)

# Book desk in all dates
for date in dates:
    print(f"selecting the date {date}...")
    # Select date
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//p[contains(@class, 'MuiTypography-root') and text()='{date}']"))
    )
    element.click()
    time.sleep(1)
    try:
        # Try to find the <span> with the desk_number text inside the <div>
        desk_number_span = div_element.find_element(By.TAG_NAME, "span")

        # Check if the desk number matches the expected value
        if desk_number_span.text == desk:
            print(f"Desk number {desk} is found. Booking...")
            # Book the desk
            try:
                book_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sc-oVpqz.lpruGk"))
                )
                book_button.click()
            except TimeoutException:
                print(f"Desk number {desk} is not able to be booked on date {date}!")
        else:
            print(f"Desk number found: {desk_number_span.text}. No action performed. Check the code.")

    except NoSuchElementException:
        print("No <span> with desk_number found inside the <div>. Skipping action.")

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6701638A5+3004357]
	(No symbol) [0x00007FF66FDF9970]
	(No symbol) [0x00007FF66FCA582A]
	(No symbol) [0x00007FF66FCF5B8E]
	(No symbol) [0x00007FF66FCF5E7C]
	(No symbol) [0x00007FF66FD3EC27]
	(No symbol) [0x00007FF66FD1BC1F]
	(No symbol) [0x00007FF66FD3BA4C]
	(No symbol) [0x00007FF66FD1B983]
	(No symbol) [0x00007FF66FCE7628]
	(No symbol) [0x00007FF66FCE8791]
	GetHandleVerifier [0x00007FF67018A00D+3161901]
	GetHandleVerifier [0x00007FF6701DE060+3506048]
	GetHandleVerifier [0x00007FF6701D400D+3465005]
	GetHandleVerifier [0x00007FF66FF50EEB+830987]
	(No symbol) [0x00007FF66FE0467F]
	(No symbol) [0x00007FF66FE009D4]
	(No symbol) [0x00007FF66FE00B6D]
	(No symbol) [0x00007FF66FDF0149]
	BaseThreadInitThunk [0x00007FFE4C2B7374+20]
	RtlUserThreadStart [0x00007FFE4CB5CC91+33]


In [163]:
### Option 2: not use cookies

# username = "meysam.aghighi@ericsson.com"
# password = "myXtremelyStrongPassword"

# # Wait to find and click on "Sign in with Microsoft" 
# button = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.CLASS_NAME, "sc-imWYAI"))
# )
# button.click()

# # Wait until the email input field is present and fill it in
# email_input = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "i0116"))
# )
# email_input.send_keys(username)  # Replace with the actual email

# # Click "Next"
# driver.find_element(By.ID, "idSIButton9").click()

# # Wait to find and click "Other ways to sign in"
# button = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "idA_PWD_SwitchToCredPicker"))
# )
# button.click()

# # Wait to find and click "use password"
# button = WebDriverWait(driver, 10).until(
#     EC.element_to_be_clickable((By.XPATH, "//div[normalize-space(text())='Use my password']"))
# )
# button.click()

# # Wait to find and fill in "password"
# password_input = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "i0118"))
# )
# password_input.send_keys(password)

# # Wait to find and click on "Sign in"
# button = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "idSIButton9"))
# )
# button.click()

In [14]:
import os
import sqlite3
from Cryptodome.Cipher import AES
import win32crypt  # Available on Windows for decrypting Chrome cookies
import json
import base64

# Path to the Cookies file in Chrome's default location
cookies_path = os.path.join(os.getenv("LOCALAPPDATA"), r"Google\Chrome\User Data\Default\Network\Cookies2")

def get_encryption_key():
    # Locate the Local State file to retrieve encryption key
    local_state_path = os.path.join(os.getenv("LOCALAPPDATA"), r"Google\Chrome\User Data\Local State")
    with open(local_state_path, "r", encoding="utf-8") as file:
        local_state = json.loads(file.read())
    # Decode the encrypted key
    encrypted_key = base64.b64decode(local_state["os_crypt"]["encrypted_key"])
    # Remove the DPAPI prefix and decrypt the key
    decrypted_key = win32crypt.CryptUnprotectData(encrypted_key[5:], None, None, None, 0)[1]
    return decrypted_key

def decrypt_cookie(encrypted_value, key):
    # AES decryption of cookies
    iv = encrypted_value[3:15]  # Initialization vector
    encrypted_value = encrypted_value[15:]
    cipher = AES.new(key, AES.MODE_GCM, iv)
    return cipher.decrypt(encrypted_value).decode("utf-8")

def fetch_cookies():
    encryption_key = get_encryption_key()
#     print(encryption_key)
    with sqlite3.connect(cookies_path) as conn:
#         conn.row_factory = sqlite3.Row  # Ensures we get BLOBs correctly
        conn.row_factory = str
        cursor = conn.cursor()
        cursor.execute("SELECT host_key, name, encrypted_value FROM cookies")
        cookies = []
        row = str
        for row in cursor.fetchall():
            host = row['host_key']
            name = row['name']
            encrypted_value = row['encrypted_value']
            
            try:
                # Ensure that encrypted_value is treated as binary data
                if isinstance(encrypted_value, bytes):
                    cookie_value = decrypt_cookie(encrypted_value, encryption_key)
                else:
                    cookie_value = f"Invalid binary data"
            except Exception as e:
                cookie_value = f"Could not decrypt: {str(e)}"
            cookies.append((host, name, cookie_value))
    
    return cookies



# Fetch and print cookies
for host, name, value in fetch_cookies():
    print(f"Host: {host}, Cookie Name: {name}, Cookie Value: {value}")


OperationalError: Could not decode to UTF-8 column 'encrypted_value' with text 'v20�I����XI���q������!�#j�F��~!����|��������PdX�����*v�>�L��(��P�jX���N�19��Q�/'

In [37]:
# Connect to the database
conn = sqlite3.connect(cookies_path)

try:
    cursor = conn.cursor()

    # Test the connection
    cursor.execute("SELECT 1")
    print("Connection is successful!")

    # Get SQLite version
    cursor.execute("SELECT sqlite_version();")
    version = cursor.fetchone()
    print(f"SQLite version: {version[0]}")

    # List all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])

    # Get schema for a specific table
    table_name = 'your_table_name'  # Replace with your actual table name
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    print(f"Schema for {table_name}:")
    for column in schema:
        print(f"Column: {column[1]}, Type: {column[2]}, Not Null: {column[3]}")

except sqlite3.Error as e:
    print(f"Error occurred: {e}")
finally:
    conn.close()


Connection is successful!
SQLite version: 3.39.4
Tables in the database:
meta
cookies
Schema for your_table_name:


In [39]:
import sqlite3

# Connect to the database
conn = sqlite3.connect(cookies_path)  # Replace with your actual database path
cursor = conn.cursor()

# 1. Check the connection
def test_connection():
    try:
        cursor.execute("SELECT 1")
        print("Connection is successful!")
    except sqlite3.Error as e:
        print(f"Error occurred: {e}")

# 2. Get column details for a given table
def get_table_columns(table_name):
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    print(f"Columns for {table_name}:")
    for column in columns:
        print(f"Column: {column[1]}, Type: {column[2]}, Not Null: {column[3]}, Default: {column[4]}")

# 3. Get row count for a given table
def get_row_count(table_name):
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    print(f"{table_name} has {row_count} rows.")

# 4. Fetch some rows (top rows) from a given table
def get_top_rows(table_name, limit=5):
    cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")
    rows = cursor.fetchall()
    print(f"Top {limit} rows from {table_name}:")
    for row in rows:
        print(row)

# 5. Perform a join between two tables
def join_tables(table1, table2, join_column):
    query = f"""
    SELECT * 
    FROM {table1} AS t1
    JOIN {table2} AS t2
    ON t1.{join_column} = t2.{join_column}
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    print(f"Joined data from {table1} and {table2}:")
    for row in rows:
        print(row)

# 6. Get summary statistics (e.g., count, average) for a column in a table
def get_summary_statistics(table_name, column_name):
    cursor.execute(f"SELECT COUNT(*), AVG({column_name}) FROM {table_name}")
    result = cursor.fetchone()
    print(f"Summary statistics for {column_name} in {table_name}:")
    print(f"Count: {result[0]}, Average: {result[1]}")

# 7. Get unique values from a column in a table
def get_unique_values(table_name, column_name):
    cursor.execute(f"SELECT DISTINCT {column_name} FROM {table_name}")
    rows = cursor.fetchall()
    print(f"Unique values in {column_name} from {table_name}:")
    for row in rows:
        print(row[0])

# Main execution
def main():
    test_connection()  # Test the database connection

    # Get column details for both tables
    get_table_columns('meta')  # Replace 'table1' with your actual table name
    get_table_columns('cookies')  # Replace 'table2' with your actual table name

    # Get row count for both tables
    get_row_count('meta')  # Replace 'table1' with your actual table name
    get_row_count('cookies')  # Replace 'table2' with your actual table name

    # Fetch some rows from both tables (top 5 rows)
    get_top_rows('meta')  # Replace 'table1' with your actual table name
    get_top_rows('cookies')  # Replace 'table2' with your actual table name

    # Perform a join between the two tables
    join_tables('meta', 'cookies', 'id')  # Replace 'id' with your actual join column

    # Get summary statistics for a column in each table
    get_summary_statistics('meta', 'column_name')  # Replace 'column_name' with the actual column name
    get_summary_statistics('cookies', 'column_name')  # Replace 'column_name' with the actual column name

    # Get unique values for a column in each table
    get_unique_values('meta', 'column_name')  # Replace 'column_name' with the actual column name
    get_unique_values('cookies', 'column_name')  # Replace 'column_name' with the actual column name

# Run the main function
main()

# Close the connection
conn.close()


Connection is successful!
Columns for meta:
Column: key, Type: LONGVARCHAR, Not Null: 1, Default: None
Column: value, Type: LONGVARCHAR, Not Null: 0, Default: None
Columns for cookies:
Column: creation_utc, Type: INTEGER, Not Null: 1, Default: None
Column: host_key, Type: TEXT, Not Null: 1, Default: None
Column: top_frame_site_key, Type: TEXT, Not Null: 1, Default: None
Column: name, Type: TEXT, Not Null: 1, Default: None
Column: value, Type: TEXT, Not Null: 1, Default: None
Column: encrypted_value, Type: BLOB, Not Null: 1, Default: None
Column: path, Type: TEXT, Not Null: 1, Default: None
Column: expires_utc, Type: INTEGER, Not Null: 1, Default: None
Column: is_secure, Type: INTEGER, Not Null: 1, Default: None
Column: is_httponly, Type: INTEGER, Not Null: 1, Default: None
Column: last_access_utc, Type: INTEGER, Not Null: 1, Default: None
Column: has_expires, Type: INTEGER, Not Null: 1, Default: None
Column: is_persistent, Type: INTEGER, Not Null: 1, Default: None
Column: priority, Typ

OperationalError: Could not decode to UTF-8 column 'encrypted_value' with text 'v20�I����XI���q������!�#j�F��~!����|��������PdX�����*v�>�L��(��P�jX���N�19��Q�/'

In [40]:
cookies_path

'C:\\Users\\eaghmey\\AppData\\Local\\Google\\Chrome\\User Data\\Default\\Network\\Cookies2'